# Lava-dl hdf5 export

In order to test the export process, we used the [Oxford example](https://lava-nc.org/lava-lib-dl/slayer/notebooks/oxford/train.html) available in the LAVA-dl repository. Since this page is focused on NIR implementations, we are not covering the training aspects, but readers are welcome to visit the LAVA-dl documentation and train their own models. To facilitate the reproducibility of the tests, we copied a trained CUBALiF dense network into the NIR repository.

It is important to mention that this `network.net` has been exported with the standard `hdf5` process in LAVA-dl, which takes the network blocks and saves them in this format.


```
class Network(torch.nn.Module):
    def __init__(self):
     ...
     ...

    def forward(self, spike):
     ...
     ...

    def export_hdf5(self, filename):
        # network export to hdf5 format
        h = h5py.File(filename, 'w')
        layer = h.create_group('layer')
        for i, b in enumerate(self.blocks):
            b.export_hdf5(layer.create_group(f'{i}'))  
```

Although the standard is `hdf5`, the block structure is not compatible with NIR graphs, so the `lava_to_nir.py` functions are required. 

# Lava-dl to NIR

Once we have the `hdf5` network from LAVA-dl, transforming it to a NIRgraph is as simple as:

In [ ]:
import lava_to_nir as lava_to_nir

lava_to_nir.convert_to_nir('network.net', 'network.nir')


If the network is compatible, it will generate the `network.nir` graph from the LAVA network.

### What's happening inside?

The first part of the function recursively inspects all layers in the LAVA graph.

In [ ]:
def convert_to_nir(net_config: PATH_TYPE, path: PATH_TYPE) -> nir.NIRGraph:
    """Load a NIR from a HDF/conn5 file."""
    with h5py.File(net_config, "r") as f:
        nir_graph = read_node(f["layer"])

Then, it maps each node to a specific function. Notice that LAVA layers include both synapses and neurons. For example, a Dense layer consists of a dense synapse plus an array of neurons of any type. For that reason, the function reads LAVA layers and splits them into synapses and neurons according to NIR. At the moment, only CUBALiF neurons are mapped.

Now, let’s dive into the CUBALiF neuron implementation. The main difference between NIR and LAVA is that NIR’s CUBALiF representation is continuous, while LAVA uses a discrete version of the dynamics to make it compatible with Loihi hardware. According to the [NIR2Lava script](https://github.com/neuromorphs/NIR/blob/main/paper/nir_to_lava.py), we decided to use `dt=1e-4` value by default.

Another important note is that LAVA-dl exports scaled hardware values to `hdf5`. However, to be more standard, we decided to export real floating-point values to NIR. This means we need to de-scale the current decay, voltage decay, and threshold values using the `scale` and ``weight_scale` factors (See [LAVA-dl CUBA-LiF](https://lava-nc.org/lava-lib-dl/slayer/neuron/neuron.html#module-lava.lib.dl.slayer.neuron.cuba)). By default, these values are `4096` and `64`.

In [ ]:
# Lava-dl exports hardware fixed tipe to hdf5. For NIR, export in floating point. 
dt      = 1e-4  # This dt value is according to nir_to_lava.py script. https://github.com/neuromorphs/NIR/blob/main/paper/nir_to_lava.py#L67
vdecay  = layer['neuron']['vDecay'][()]  / 4096 # Save the value in NIR as floating point
idecay  = layer['neuron']['iDecay'][()]  / 4096 # Save the value in NIR as floating point
thr     = layer['neuron']['vThMant'][()] / 64   # Save the value in NIR as floating point
tau_mem = dt/float(vdecay) if vdecay != 0 else np.inf
tau_syn = dt/float(idecay) if idecay != 0 else np.inf
shape   = layer['weight'].shape[0]
r       = tau_mem/dt # no scaling of synaptic current
w_in    = tau_syn/dt # no scaling of synaptic voltage

return nir.CubaLIF(
    tau_syn=np.full(shape, tau_syn),
    tau_mem=np.full(shape, tau_mem),
    r=np.full(shape, r),  
    v_leak=np.full(shape, 0.),  # currently no bias in Loihi's neurons
    v_threshold=np.full(shape, thr),
    w_in=np.full(shape,w_in),  
    v_reset=np.full(shape,0.) # LAVA-DL CUBA LiF always reset to 0
)

Another difference between LAVA graphs and NIR graphs is the naming convention. LAVA uses `input_#` and `output_#` (where `#` is any number), while NIR uses only `input` and `output`. Moreover, to make it compatible with the `hdf5` standard, all strings need to be converted to bytes. The function `normalize_nir_graph()` performs this conversion.

In [ ]:
nir_graph = normalize_nir_graph(nir_graph, to_bytes_in_edges=True)

After exporting the NIR graph, it can be converted back to LAVA-dl and plotted to compare both implementations.

In [ ]:
import nir
from lib.nir_to_lava import ImportConfig, LavaLibrary, import_from_nir

import_config = ImportConfig(library_preference=LavaLibrary.LavaDl)
nir_graph = nir.read('/home/garciaal/neuromorphics/Trained/network.nir')
nir_net = import_from_nir(nir_graph, import_config)

Then, you can use the DataLoader and the process in the [Oxford example](https://lava-nc.org/lava-lib-dl/slayer/notebooks/oxford/train.html) to generate the plots and compare them with the NIR network. The code below shows an example using the LAVA-dl tools, but the Oxford dataset is missing.

In [ ]:
import lava.lib.dl.slayer as slayer
from lava.lib.dl import netx
import torch
import matplotlib.pyplot as plt

lava_net = netx.hdf5.Network(net_config='/home/garciaal/neuromorphics/Trained/network.net')

device = torch.device('cpu')
nir_net.to(device)
lava_net.to(device)

# Missing input dataset. Look for LAVA-dl examples
output_lava = lava_net(input.to(device))
output_nir = nir_net(input.to(device))
event_nir = slayer.io.tensor_to_event(output_nir.cpu().data.numpy())
event_lava = slayer.io.tensor_to_event(output_lava.cpu().data.numpy())

# Calculate loss. This has to be 0
error = slayer.loss.SpikeTime(time_constant=10, filter_order=2).to(device)
loss_nir = error(output_nir, output_lava)
print(f'nir loss to LAVA: {loss_nir}')



```nir loss to LAVA: 0```

And finally, plot the results

In [ ]:

plt.figure(figsize=(200, 200))
plt.plot(event_lava.t, event_lava.x, '.', markersize=12, label='lava network')
plt.plot(event_nir.t, event_nir.x, '.', label='nir network')
plt.xlabel('time [ms]')
plt.ylabel('neuron')
plt.legend()
plt.savefig('nir_output.png')

In [ ]:
from IPython.display import Image

Image(filename='nir_output.png')

: 

## Main limitations and things to improve

1. Using `dt=1e-4` by default according to the `nir_to_lava.py` script. This needs to be confirmed with the LAVA developers.

2. Only tested with 1D Dense CUBA LiF neurons. According to the code, more work is needed for 2D Dense layers.

3. Different neuron types were not tested. Only CUBA LiF is implemented.

4. Find a way to automate edge string correction. Currently, it is done manually:

   ```
   nir_graph.edges[3]=(b'input',0) 
   nir_graph.edges[4]=(3,b'output')
   ```
5. NIR graphs converted to LAVA-dl do not include delays when executing networks. Therefore, NIR-converted networks will generate an output without any network delay. Be careful when comparing with LAVA, since the loss may be different from 0 even if the plots look the same.